<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/generative/StableDiffusion_image_inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 環境設定

# ライブラリのインストール
!pip install git+https://github.com/huggingface/diffusers.git
!pip install transformers scipy ftfy

# インポート
from diffusers import StableDiffusionInpaintPipeline
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from google.colab import files
import numpy as np
import torch
from torch import autocast

In [ ]:
#@title モデルの設定

# 学習済みモデルのダウンロード
access_token = '' #@param {type:'string'}
model_name = "runwayml/stable-diffusion-inpainting"  # 'CompVis/stable-diffusion-v1-4'
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    model_name,
    revision='fp16', 
    torch_dtype=torch.float16,
    use_auth_token=access_token
).to('cuda')

In [ ]:
#@title ベース画像の読込

# 画像のアップロード
uploaded = files.upload()

# ファイル名の設定
image_file = list(uploaded.keys())[0]

# ベース画像の読込
original_image = Image.open(image_file).convert('RGB')
init_image = original_image.resize((512, 512))

# 修復領域の指定
x1 = 205  #@param {type:"integer"}
y1 = 5    #@param {type:"integer"}
x2 = 320  #@param {type:"integer"}
y2 = 170  #@param {type:"integer"}
mask = init_image.copy()
draw = ImageDraw.Draw(mask)
draw.rectangle([(x1, y1), (x2, y2)], outline=(0, 255, 0), width=3)
mask

In [ ]:
#@title 画像修復

# マスク画像の作成
draw.rectangle([(0, 0), (512, 512)], outline=(0, 0, 0), fill=(0, 0, 0))
draw.rectangle([(x1, y1), (x2, y2)], outline=(255, 255, 255), fill=(255, 255, 255))

# プロンプトの設定
prompt = 'Albert Einstein' #@param {type:'string'}

# 出力ディレクトリの作成
!mkdir -p outputs

# 生成画像数の設定
num = 50

# 画像の修復
plt.figure(figsize=(30, 30))
for i in range(num):
    plt.subplot(10, 5, i + 1)
    with autocast('cuda'):
        image = pipe(prompt=prompt, image=init_image, mask_image=mask).images[0]
    image.save(f'outputs/{i:03}.png')
    plt.imshow(image)
    plt.axis('off')
    plt.title(i)
    plt.show;